# Building Database for crime reports
The goal of this project is to create a database named *crimes_db* with a table *boston_crimes* with appropriate datatypes for storing the data from the boston.csv file. We'll be creating the table inside a schema named *crimes*. We'll also create the *readonly* and *readwrite* groups with the appropriate privileges. Finally, we will also need to create one user for each of these groups.

In [1]:
import psycopg2

conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True
# creating crimes_db database
cur =  conn.cursor()
cur.execute('DROP DATABASE IF EXISTS crimes_db;')
cur.execute('CREATE DATABASE crimes_db;')
conn.close()

In [2]:
conn = psycopg2.connect(dbname="crimes_db", user="dq")
conn.autocommit = True
# creating crimes schema in crimes_db
cur =  conn.cursor()
cur.execute('DROP SCHEMA IF EXISTS crimes;')
cur.execute('CREATE SCHEMA crimes;')
#conn.autocommit = False

In [3]:
import csv
# reading boston.csv file
with open('boston.csv', 'r') as f:
    rows = list(csv.reader(f))
    col_header = rows[0]
    first_row = rows[1]

In [4]:
col_header, first_row

(['incident_number',
  'offense_code',
  'description',
  'date',
  'day_of_the_week',
  'lat',
  'long'],
 ['1',
  '619',
  'LARCENY ALL OTHERS',
  '2018-09-02',
  'Sunday',
  '42.35779134',
  '-71.13937053'])

In [5]:
# number of rows of data in CSV file
len(rows[1:]) 

298329

In [6]:
def get_col_value_set(csv_filename, col_index):
    '''
    args: 
    csv_filename- name of a CSV file
    col_index- index of a column of that CSV file
    
    returns:
    a Python set that contains all distinct values from that column
    '''
    col_set = set()
    with open(csv_filename, 'r') as f:
        rows = list(csv.reader(f))
        for row in rows[1:]:
            col_set.add(row[col_index])
    return col_set

In [7]:
# calculating no of values in a column
for i in range(len(col_header)):
    values = get_col_value_set('boston.csv', i)
    print(col_header[i], len(values))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


In [8]:
# finding longest value length in a description column
max_len = 0
for value in get_col_value_set('boston.csv', 2):
    max_len = max(max_len, len(value))
print('maximum length of a value in description column:', max_len)

maximum length of a value in description column: 58


In [9]:
print(col_header)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [10]:
# creating the enumerated datatype for representing the weekday
cur.execute('''
    CREATE TYPE day_of_week_enum AS ENUM(
        'Sunday', 'Monday', 'Tuesday', 'Wednesday', 
        'Thursday', 'Friday', 'Saturday' );''')

# creating boston_crimes table in crimes schema
cur.execute(''' 
    CREATE TABLE crimes.boston_crimes(
                incident_number INTEGER PRIMARY KEY,
                offense_code INTEGER,
                description VARCHAR(100),
                date DATE,
                day_of_the_week day_of_week_enum,
                lat DECIMAL,
                long DECIMAL);''')


In [11]:
# loading boston.csv data into boston_crimes table
with open('boston.csv', 'r') as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

In [12]:
# revoking public privileges
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crimes_db FROM public;")

In [13]:
# creating readonly and granting necessary accesses
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimeS TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

In [14]:
# creating readwrite and granting necessary accesses
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimeS TO readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

In [16]:
# creating data_analyst user and assinging to readonly group
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

# creating data_scientist user and assinging to readwrite group
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")